<a href="https://colab.research.google.com/github/varunsawhney8/Projects/blob/main/Fdeployedfinbert1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install streamlit

In [2]:
!pip install pyngrok==4.1.1

  Created wheel for pyngrok: filename=pyngrok-4.1.1-py3-none-any.whl size=15984 sha256=b87fc2aa9fc1702ea77f42fb2c960d0c2d13f33a8933fd3ba04e95b7d11cff86
  Stored in directory: /root/.cache/pip/wheels/b1/d9/12/045a042fee3127dc40ba6f5df2798aa2df38c414bf533ca765
Successfully built pyngrok


In [3]:
!pip install transformers

     |████████████████████████████████| 2.8 MB 9.4 MB/s 
     |████████████████████████████████| 50 kB 5.5 MB/s 
     |████████████████████████████████| 895 kB 55.6 MB/s 
     |████████████████████████████████| 636 kB 57.4 MB/s 
     |████████████████████████████████| 3.3 MB 41.2 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [36]:
%%writefile app.py

# Project: Sentiment Analysis 
import pandas as pd
import string
import re
from os import listdir
from nltk.corpus import stopwords
import numpy as np
import nltk
import requests
import os
import streamlit as st
import altair as alt


######## output ###############################################################

#company_nse_2.to_csv("Reliance_Nse_data2.csv",encoding = "utf-8")

# b) Load dataset
import pandas as pd
path='/content/Reliance_Final_Ndtv_data4.xlsx'

data=pd.read_excel(path)

data=data[::-1].reset_index()
data.drop(columns=['index'],axis=True,inplace=True)

path='/content/Reliance_Nse_data2.csv'

data1=pd.read_csv(path)

# 2. Summarize Data
# a) Descriptive statistics
# b) Data visualizations

data.drop(columns=['Media','Article','Keywords'],inplace=True,axis=1)

# 3. Prepare Data
# a) Data Cleaning
# b) Feature Selection
# c) Data Transforms
def clean_data(data):
    
    list1=[]
    list1=data['Title']
    import re
    import nltk
    nltk.download('stopwords')
    
    for i in range(len(list1)):               
        tokens = list1[i].split()
        # prepare regex for char filtering
        re_punc = re.compile('[%s]' % re.escape(string.punctuation))
        # remove punctuation from each word
        tokens = [re_punc.sub('', w) for w in tokens]
        # remove remaining tokens that are not alphabetic
        tokens = [word for word in tokens if word.isalpha()]
        # filter out stop words
        stop_words = set(stopwords.words('english'))
        tokens = [w for w in tokens if not w in stop_words]
        # filter out short tokens
        tokens = [word for word in tokens if len(word) > 1]
        list1[i] = ' '.join(tokens)
    data['Title']=list1
    return data

    
#del list1

# 4. Evaluate Algorithms
# a) Split-out validation dataset- train_test_split
# b) Test options and evaluation metric- Confusion matrix, accuracy, precision,recall, f measure
# c) Spot Check Algorithms- Unsupervised [Textblob, Vader, LM Dictionary,FINBERT], Supervised-[Bert]
# d) Compare Algorithms

# =============================================================================
# """ MODEL:**
# 1. FINBERT
# """
# =============================================================================
#!pip install transformers
from transformers import pipeline

def model_sentiment(data):
    classifier = pipeline('sentiment-analysis', model="ProsusAI/finbert")
    list3=[]
    for i in range(len(data['Title'])):
        a=(classifier(data['Title'][i]))
        a=a[0]
        a=list(a.values())[0]
        label_f={'positive':1,'negative':2,'neutral':0}
        list3.append(label_f[a])
    data['1day_sentiment']=list3
    del list3
    return (data)



# """#4. Business Problem Analysis
#     A. Sentiment Meter
#     B. Price Movement analysis based on Sentiment meter
# """
# 
# 
# =============================================================================

def business(data,data1):
    # Groupby based on dates as several news articles are published on a particular day

    aggregation_functions = {'Title': 'first', '1day_sentiment': 'max'}
    data = data.groupby(data['Date']).aggregate(aggregation_functions)
    data.reset_index( inplace=True)
    data['Date']=pd.to_datetime(data['Date']).dt.date
    data['1day_sentiment']=data['1day_sentiment'].astype('float')
    
    # Calculating Exponential Weighted average for 3,7,15 days sentiments

    data['3day_sentiment'] = round(data['1day_sentiment'].ewm(span=3).mean())
    data['7day_sentiment'] = round(data['1day_sentiment'].ewm(span=7).mean())
    data['15day_sentiment'] = round(data['1day_sentiment'].ewm(span=15).mean())
    data.head()
    
    # Converting date time format
    data1['Date']=pd.to_datetime(data1['Date']).dt.date
    
    # Calculating percentage change w.r.t closing price

    data1['perc_change1'] =data1['Close'].pct_change(periods=1)*100

    data2=data.drop(columns=['Title'],axis=1)
    
    # Imputation of perc_change
    data2["perc_change1"]=[np.nan for i in range(len(data))]
    
    label={0: 'Neutral',1:'Positive',2:'Negative'}
    data2['1day_sentiment']=[label[data2['1day_sentiment'][i]]for i in range(len(data2))]
    data2['3day_sentiment']=[label[data2['3day_sentiment'][i]]for i in range(len(data2))]
    data2['7day_sentiment']=[label[data2['7day_sentiment'][i]]for i in range(len(data2))]
    data2['15day_sentiment']=[label[data2['15day_sentiment'][i]]for i in range(len(data2))]
    
    
    for i in range(len(data2)):
      for j in range(len(data1)):
        if data['Date'][i]==data1['Date'][j]:
          data2['perc_change1'][i]=data1['perc_change1'][j+1]
      count=0
      if type(data2['perc_change1'][i]) and pd.isna(data2['perc_change1'][i]):
        for j in range(len(data1)):
          if data2['Date'][i]>data1['Date'][j]:
            count+=1
        data2['perc_change1'][i]=data1['perc_change1'][count]
    
    data2['perc_change1'][0]=np.nan
    
    data2['Close']=[np.nan for i in range(len(data))]
    
    for i in range(len(data2)):
      for j in range(len(data1)):
        if data['Date'][i]==data1['Date'][j]:
          data2['Close'][i]=data1['Close'][j+1]
      count=0
      if type(data2['Close'][i]) and pd.isna(data2['Close'][i]):
        for j in range(len(data1)):
          if data2['Date'][i]>data1['Date'][j]:
            count+=1
        data2['Close'][i]=data1['Close'][count]

    # To Study how many investment oppurtunities were profitable, loss making
    data4=pd.DataFrame(columns=["Description","Total Opportunities","Profit(%)","Loss(%)","No Profit No Loss(%)"])
    desc=[]
    total1=[]
    profit1=[]
    loss1=[]
    npnl1=[]
    col=['1day_sentiment', '3day_sentiment', '7day_sentiment','15day_sentiment']
    for j in col:
      profit=0
      loss=0
      npnl=0
      for i in range(len(data2)):
          if data2[j][i]=='Negative' and  data2['perc_change1'][i]<0:
            profit+=1
          elif data2[j][i]=='Negative' and  data2['perc_change1'][i]>0:
            loss+=1
            
          if data2[j][i]=='Positive' and  data2['perc_change1'][i]>0:
            profit+=1
          elif data2[j][i]=='Positive' and  data2['perc_change1'][i]<0:
            loss+=1
          if data2[j][i]=='Neutral':
            npnl+=1
      total= profit + loss +npnl
      desc.append(j)
      total1.append(total)
      profit1.append(profit/total*100)
      loss1.append(loss/total*100)
      npnl1.append(npnl/total*100)
    
    data4["Description"]=["Rolling 1 day-News Day Event","Rolling 3 day-News Day Event","Rolling 7 day-News Day Event ","Rolling 15 day-News Day Event"]
    data4["Total Opportunities"]=total
    data4['Profit(%)']=profit1
    data4['Loss(%)']=loss1
    data4['No Profit No Loss(%)']=npnl1
    
    data_1=data.drop(columns=["Title"],axis=1)
    data_f1=data_1.iloc[len(data_1)-1,]
    label={0.0: 'Neutral',1.0:'Positive',2.0:'Negative'}
    f_data=pd.DataFrame(columns=["Description", "Sentiment"])
    f_data["Description"]=["Rolling 1 day-News Day Event","Rolling 3 day-News Day Event","Rolling 7 day-News Day Event ","Rolling 15 day-News Day Event"]
    f_data["Sentiment"]=[label[data_f1[i]] for i in range(1,len(data_f1),1)]
    
    return f_data,data4,data_f1


#A. Cleaning the data
data=clean_data(data)

#B. Predicting Sentiments
data=model_sentiment(data)

#C. Measuring Backtesting Investment Oppurtunities
f_data,data4,data_f1= business(data, data1)

# Streamlit Deployment Code

companies=['1:INFOSYS LTD.', '2:TATA CONSULTANCY SERVICES LTD.' , '3:RELIANCE INDUSTRIES LTD.',
 '4:ICICI BANK LTD.', '5: HDFC BANK LTD.', '6: HCL TECHNOLOGIES LTD.',
 '7:BHARTI AIRTEL LTD.', '8: INDUSIND BANK LTD.', '9: STATE BANK OF INDIA',
'10: LARSEN & TOUBRO LTD.', '11: TECH MAHINDRA LTD.', '12: AXIS BANK LTD.',
'13: ITC LTD.', '14: BAJAJ AUTO LTD.', '15: OIL AND NATURAL GAS CORPORATION LTD.',
'16: TATA STEEL LTD.', '17: NTPC LTD.', '18: MAHINDRA & MAHINDRA LTD.',
'19: ASIAN PAINTS LTD.', '20: POWER GRID CORPORATION OF INDIA LTD.',
'21: BAJAJ FINSERV LTD.', '22: TITAN COMPANY LTD.', '23: NESTLE INDIA LTD.',
'24: ULTRATECH CEMENT LTD.' , '25: SUN PHARMACEUTICAL INDUSTRIES LTD.',
'26:BAJAJ FINANCE LTD.', '27: MARUTI SUZUKI INDIA LTD.', '28: HOUSING DEVELOPMENT FINANCE CORP. LTD',
'29: HINDUSTAN UNILEVER LTD.', '30: KOTAK MAHINDRA BANK LTD.']

from PIL import Image
st.set_page_config(page_title="NLP applications in Finance", page_icon=None, layout='wide', initial_sidebar_state='expanded')
image = Image.open('/content/IMG2.png')
st.image(image, use_column_width=True)


#st.sidebar.subheader("About us")
st.sidebar.title("About us")
st.sidebar.write('''
Innodatatics is actively transformed by its highly extreme entities with various decades of realm 
expertise among its representatives. Our R&D team advances to conceive by collaborating with its alma maters 
in finding solution industry-complicated issues.''')

st.sidebar.subheader("Project")
st.sidebar.write("Measuring Impact of Financial News on Stock Market")
select = st.sidebar.selectbox( "Choose a company?",(companies))
st.sidebar.subheader("Contact us")
st.sidebar.write("Innodatatics Inc")
st.sidebar.write("[Website](https://innodatatics.ai)")
st.sidebar.write("© Copyrights 2021 Innodatatics")



#select = st.selectbox
x=int(select[0])


st.subheader("Selected Company:")
st.write(select[2:])

# PIE CHART CODE

import matplotlib.pyplot as plt
labels = ['Neutral', 'Positive','Negative']
sizes = [data['1day_sentiment'].value_counts()[0],data['1day_sentiment'].value_counts()[1],data['1day_sentiment'].value_counts()[2]]
# Plot
fig, ax = plt.subplots(figsize=(4,4))
#st.subheader('News Classification based on sentiment Analysis')
ax.pie(sizes, labels=labels, autopct='%1.1f%%', shadow=False, startangle=140)
plt.title('News type based on sentiment Analysis')
from io import BytesIO
buf1 = BytesIO()
fig.savefig(buf1, format="png")
#st.image(buf1)

# Word CLoud
pos=[]
neg=[]
neutral=[]
          
# Data Divide into segments
          
for i in range(len(data)):
    if data['1day_sentiment'][i]=='negative':
        pos.append(data['Title'][i])
    elif data['1day_sentiment'][i]=='positive':
        neg.append(data['Title'][i])
    else:
        neutral.append(data['Title'][i])

#             
from collections import Counter
vocab=Counter()          
#             
vocab_pos=[]
#             
for i in range(len(pos)):
    tokens=pos[i].split()
    for j in tokens:
        vocab_pos.append(j)
#             
vocab_neg=[]
#             
for i in range(len(neg)):
    tokens=neg[i].split()
    for j in tokens:
        vocab_neg.append(j)
  #             
vocab_neutral=[]
#             
for i in range(len(neutral)):
    tokens=neutral[i].split()
    for j in tokens:
        vocab_neutral.append(j)
#                          
vocab.update(vocab_pos)
vocab.update(vocab_neutral)
vocab.update(vocab_neg)          
vocab.most_common(50)
#
from wordcloud import WordCloud, STOPWORDS
#             
other_stopwords_to_remove = ['\\n', 'n', '\\', '>', 'nLines', 'nI',"n'", "hi"]
STOPWORDS = STOPWORDS.union(set(other_stopwords_to_remove))
stopwords = set(STOPWORDS)
text = str(vocab)
wordcloud = WordCloud(width = 1800, height = 1800, background_color ='white', 
                          max_words=200, stopwords = stopwords, min_font_size = 10).generate(text)

fig, ax = plt.subplots(figsize=(4,4))
ax.imshow(wordcloud, interpolation='bilinear')
plt.title("News based word cloud")
plt.axis("off")
from io import BytesIO
buf2 = BytesIO()
fig.savefig(buf2, format="png")
#st.image(buf2)

images = [buf1,buf2]
st.image(images, use_column_width=False)

st.subheader("Current News Based Sentiment Report")
st.write("Last News Day Recoded : ",data_f1[0])
#f_data.set_index('Description')
st.write(f_data)
st.subheader("Back Testing News Based Investment Oppurtunities")
st.subheader("Square off trade within  next trading day ")
st.write(data4)


Overwriting app.py


In [6]:
!ls

app.py	  Reliance_Final_Ndtv_data4.xlsx  sample_data
nlp2.PNG  Reliance_Nse_data2.csv


In [7]:
!ngrok authtoken ./ngrok authtoken 1xHDQ0ZVGhTEWf2WvFJWvwgCW9U_7MWDgdeCydvLzgnNvBSa4

NAME:
   authtoken - save authtoken to configuration file

USAGE:
   ngrok authtoken [command options] [arguments...]

DESCRIPTION:
   The authtoken command modifies your configuration file to include
   the specified authtoken. By default, this configuration file is located
   at $HOME/.ngrok2/ngrok.yml

   The ngrok.com service requires that you sign up for an account to use
   many advanced service features. In order to associate your client with
   an account, it must pass a secret token to the ngrok.com service when it
   starts up. Instead of passing this authtoken on every invocation, you may
   use this command to save it into your configuration file so that your
   client always authenticates you properly.

EXAMPLE:
    ngrok authtoken BDZIXnhJt2HNWLXyQ5PM_qCaBq0W2sNFcCa0rfTZd

OPTIONS:
   --config 		save in this config file, default: ~/.ngrok2/ngrok.yml
   --log "false"	path to log file, 'stdout', 'stderr' or 'false'
   --log-format "term"	log record format: 'term', 'logfmt',

In [ ]:
!ngrok

NAME:
   ngrok - tunnel local ports to public URLs and inspect traffic

DESCRIPTION:
    ngrok exposes local networked services behinds NATs and firewalls to the
    public internet over a secure tunnel. Share local websites, build/test
    webhook consumers and self-host personal services.
    Detailed help for each command is available with 'ngrok help <command>'.
    Open http://localhost:4040 for ngrok's web interface to inspect traffic.

EXAMPLES:
    ngrok http 80                    # secure public URL for port 80 web server
    ngrok http -subdomain=baz 8080   # port 8080 available at baz.ngrok.io
    ngrok http foo.dev:80            # tunnel to host:port instead of localhost
    ngrok http https://localhost     # expose a local https server
    ngrok tcp 22                     # tunnel arbitrary TCP traffic to port 22
    ngrok tls -hostname=foo.com 443  # TLS traffic for foo.com to port 443
    ngrok start foo bar baz          # start tunnels from the configuration file

VERSI

In [8]:
from pyngrok import ngrok

In [9]:
!streamlit run app.py &>/dev/null&

In [10]:
!pgrep streamlit

264


In [11]:
!ngrok http	./ngrok http 8501

NAME:
   http - start an HTTP tunnel

USAGE:
   ngrok http [command options] [arguments...]

DESCRIPTION:
   Starts a tunnel listening for HTTP/HTTPS traffic with a specific hostname.
   The HTTP Host header on incoming public requests is inspected to
   determine which tunnel it matches.

   HTTPS endpoints terminate TLS traffic at the ngrok server using the
   ngrok.io certificates. The decrypted, HTTP traffic is then forwarded
   through the secure tunnel and then to your local server. If you don't want
   your TLS traffic to terminate at the ngrok server, use a TLS or TCP tunnel.

EXAMPLES:
   ngrok http 8080                      # forward ngrok.io subdomain to port 80
   ngrok http example.com:9000          # forward traffic to example.com:9000
   ngrok http -subdomain=bar 80         # request subdomain name: 'bar.ngrok.io'
   ngrok http -hostname=ex.com 1234     # request tunnel 'ex.com' (DNS CNAME)
   ngrok http -auth='falken:joshua' 80  # enforce basic auth on tunnel endpoint
 

In [12]:
public_url = ngrok.connect(port='8501')
public_url

'http://7508-34-83-103-135.ngrok.io'

In [ ]:
!kill 265